How to create the environment (replace neat_test4 with your env name):

- conda create -n neat_test4 python=3.10 gym ipykernel pyglet
- conda activate neat_test4
- pip install neat-python
- pip install -e "<path_to_gym_location>\gym-sokoban"
- python -m ipykernel install --user --name neat_test4 --display-name "Python (neat_test4)"
- pip install graphviz

### Initial setups

In [1]:
import neat
import gym
import gym_sokoban
import pyglet
from pyglet import clock
import numpy as np
import pickle
import time
import logging
from neat.reporting import StdOutReporter
import random
import visualize
import graphviz
import os


In [2]:


## Custom rendering setup if gym's rendering is not available
class Viewer:
    def __init__(self, width, height):
        self.window = pyglet.window.Window(width, height)
        self.image = None
        self.window.on_draw = self.on_draw

    def render(self, image):
        self.image = pyglet.image.ImageData(image.shape[1], image.shape[0], 'RGB', image.tobytes(), pitch=image.shape[1] * -3)
        self.window.dispatch_event('on_draw')

    def on_draw(self):
        if self.image:
            self.window.clear()
            self.image.blit(0, 0)

In [3]:
# Custom reporter class
class CustomReporter(StdOutReporter):
    def __init__(self, show_species_detail, config_filename):
        super().__init__(show_species_detail)
        self.start_time = time.time()
        self.config_filename = config_filename
    
    def end(self):
        runtime = time.time() - self.start_time
        logging.info(f'Total runtime: {runtime:.2f} seconds')
    
    def post_evaluate(self, config, population, species_set, best_genome):
        super().post_evaluate(config, population, species_set, best_genome)
        
        # Log population's average fitness
        total_fitness = sum(genome.fitness for genome in population.values())
        avg_fitness = total_fitness / len(population)
        logging.info(f'Population\'s average fitness: {avg_fitness}')
        
        # Log adjusted fitness score
        adjusted_fitness = []
        for species_id, species in species_set.species.items():
            for genome_id in species.members:
                genome = population[genome_id]
                adjusted_fitness.append(genome.fitness / len(species.members))
        avg_adjusted_fitness = sum(adjusted_fitness) / len(adjusted_fitness)
        logging.info(f'Population\'s average adjusted fitness: {avg_adjusted_fitness}')
        
        # Log best genome information
        logging.info(f'\nBest genome:\nKey: {best_genome.key}\nFitness: {best_genome.fitness}')
        logging.info(f'Nodes:')
        for node_key, node in best_genome.nodes.items():
            logging.info(f'\t{node_key} {node}')
        logging.info(f'Connections:')
        for conn_key, conn in best_genome.connections.items():
            logging.info(f'\t{conn_key} {conn}')
        
        # Log configuration file content
        if self.config_filename:
            try:
                with open(self.config_filename, 'r') as f:
                    config_content = f.read()
                    logging.info(f'Config File:\n{config_content}')
            except FileNotFoundError:
                logging.warning(f'Config file "{self.config_filename}" not found.')

        # Log timestamp
        logging.info(f'Timestamp: {time.strftime("%Y-%m-%d %H:%M:%S")}')


In [4]:
# Initialize logging
logging.basicConfig(filename='neat_log.txt', level=logging.INFO, format='%(message)s')

#### Configs

In [5]:
# Load configuration.
config_filename = 'config-feedforward_v03'
config = neat.Config(neat.DefaultGenome, 
                     neat.DefaultReproduction, 
                     neat.DefaultSpeciesSet, 
                     neat.DefaultStagnation, 
                     config_filename)

# Check if a checkpoint exists
checkpoint_file = r'D:\Education\AI\Machine_Learning_Practice\Summer School 2024\Sokoban-SS2024\NEAT\run_config_03\neat-checkpoint-v03-123'  # Replace with your checkpoint filename

if os.path.isfile(checkpoint_file):
    # Load the checkpoint
    p = neat.Checkpointer.restore_checkpoint(checkpoint_file)
else:
    # Create the population if no checkpoint exists
    p = neat.Population(config)

# Add reporters to show progress in the terminal and log to file.
p.add_reporter(neat.StdOutReporter(True))
custom_reporter = CustomReporter(True, config_filename)
p.add_reporter(custom_reporter)
stats = neat.StatisticsReporter()
p.add_reporter(stats)
p.add_reporter(neat.Checkpointer(1, filename_prefix='neat-checkpoint-v03-'))

file_name = 'winner_test_03.pkl'


# Define episode and timestep parameters
num_episodes = 1
timesteps_per_episode = 40

current_episode = 0
current_timestep = 0

min_reward = -10

# param used to mutate a step
epsilon = 0.05

### Classes and methods

#### Preprocessing inputs/outputs

In [6]:
def process_observation(environment, obs):
        
    # Convert the observation to RGB frame or custom observation
    arr_walls, arr_goals, arr_boxes, arr_player = environment.render(mode='raw')

    # Initialize the combined array with walls (1s)
    combined = np.ones_like(arr_walls)
    
    # Set empty fields (0s)
    combined[arr_walls == 0] = 0
    
    # Set targets (3s)
    combined[arr_goals == 1] = 3
    
    # Set boxes (2s)
    combined[arr_boxes == 1] = 2
    
    # Set boxes on targets (4s)
    combined[(arr_boxes == 1) & (arr_goals == 1)] = 4
    
    # Set player position (5s)
    combined[arr_player == 1] = 5

    # Flatten the array
    flat_array = combined.flatten()
    
#     print("Flat array: ", flat_array)
#     print("Flat array shape: ", flat_array.shape)

    # Output the flattened array
    return flat_array



def process_state(state):
# Processes the initial state of env.reset()


    # Initialize the combined array with walls (0s)
    combined = np.ones_like(state[0])
    
    # Set empty fields (1s)
    combined[state[0] == 0] = 0

    # Set targets (3s)
    combined[state[1] == 1] = 3

    # Set boxes (2s)
    combined[state[2] == 1] = 2

    # Set boxes on targets (4s)
    combined[(state[2] == 1) & (state[1] == 1)] = 4

    # Set player position (5s)
    combined[state[3] == 1] = 5

    # Flatten the array
    flat_array = combined.flatten()
    
#     print("Flat array: ", flat_array)
#     print("Flat array shape: ", flat_array.shape)

    # Output the flattened array
    return flat_array

In [7]:
def map_action(output):
    return np.argmax(output)

### Run Neat logic

In [8]:
# Start the game
env = gym.make('Sokoban-small-v1')
# generate the level in the initial stage (env.reset) 
env.reset()


# # OPTIONAL
viewer = Viewer(160, 160)  # Adjust the size according to your environment
ACTION_LOOKUP = env.unwrapped.get_action_lookup()



def eval_genomes(genomes, config):

    global num_episodes, timesteps_per_episode, current_episode, current_timestep, min_reward

    # FOR EACH GENOME
    for genome_id, genome in genomes:
               
        # generate the neural network based on the config provided
        net = neat.nn.FeedForwardNetwork.create(genome, config)
    
        # define the initial fitness of the genome
        genome.fitness = 0.0
    
        # define episodes rewards (list) idea is to keep the fitness scores of all episodes and take the max
        episodes_rewards = []
        
        # FOR EACH EPISODE
        for episode in range(num_episodes):
            
            # Episode reward = 0  
            ep_reward = 0
            
            # reset the game state to the initial phase
            initial_state = env.reset()

            # map inputs suitable for the Neural Network (initial state of the game as flatten array, e.g. with length 49, created by the initial 7x7 grid)
            # used as an input layer in the Neural Network
            initial_inputs = process_state(initial_state)
            
            game_state_after_step = initial_inputs
            
            # FOR EACH STEP
            for step in range(timesteps_per_episode):
                
               
                # calculate probabilities for each output to be selected
                action_prob = net.activate(game_state_after_step)
                
                # select the action, based on the output's probabilities of being selected
                action = map_action(action_prob)
                
                # ADD CUSTOM MUTATION IN THE STEPS (at a random event, the alg could chose a random step)
                
                if random.random() < epsilon:
                    action = env.action_space.sample()
                else:
                    # select the action, based on the output's probabilities of being selected
                    action = map_action(action_prob)
                
#                 # RANDOM ACTION (replace with the genome)
#                 action = env.action_space.sample()

                # make the move in the game and output game state + info + reward
                observation, reward, done, info = env.step(action)
                
#                 # TODO (MAYBE) !!!! Adjust the fitness function
                
#                 # 1) USE THE info to extract information about the move and change the reward accordingly
                
                # penalizes the passiveness of the player; rewards movement of boxes more
                if not info["action.moved_player"]:
                    reward = -0.5                
                elif info["action.moved_box"]:
                    reward = 0.2
#                 elif ....
                
                game_state_after_step = process_observation(environment=env, obs=observation)

                # IMAGE STUFF
                image = env.render(mode='rgb_array')
                viewer.render(image)

#                 # PRINT INFO
                logging.info(f'Population\'s steps info: {(ACTION_LOOKUP[action], reward, done, info)}')

                # POPULATE THE Episode reward +=
                # if the game didn't end on this step
                if not done:
                    ep_reward += reward
                    current_timestep += 1
                    
                # if the game ended
                else:
#                     current_timestep = 0
#                     current_episode += 1
                    print(f"I BEAT THE STUPID GAME!!! Happend on step {current_timestep}")
                    break
    
                    # OR BREAK

            # add the accumulated rewards for this episode into a list (episodes_rewards)   
            episodes_rewards.append(ep_reward)

        # choose the best performance on an episode for the genome
        genome.fitness = max(episodes_rewards)

#         print("All episodes finished. Closing window.")
        viewer.window.close()  # Close the Pyglet window explicitly
        


# Run until a solution is found. The number indicates the max number of generations to be produced
winner = p.run(eval_genomes, 50)


# Display the winning genome.
print('\nBest genome:\n{!s}'.format(winner))

# SAVE THE WINNER GENOME
with open("winner.pkl", "wb") as f:
    pickle.dump(winner, f)

# Show output of the most fit genome against training data.
print('\nOutput:')
winner_net = neat.nn.FeedForwardNetwork.create(winner, config)

# Save the winner
with open(file_name, 'wb') as f:
    pickle.dump(winner, f)


C:\Users\Mitko\.conda\envs\neat_test3\lib\site-packages\gym\utils\passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
C:\Users\Mitko\.conda\envs\neat_test3\lib\site-packages\gym\utils\passive_env_checker.py:187: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
C:\Users\Mitko\.conda\envs\neat_test3\lib\site-packages\gym\utils\passive_env_checker.py:199: UserWarning: WARN: The result returned by `env.reset()` should be `(obs, info)` by default, , where `obs` is a observation and `info` is a dictionary containing additional information.
  logger.warn(



 ****** Running generation 123 ****** 


 ****** Running generation 123 ****** 



C:\Users\Mitko\.conda\envs\neat_test3\lib\site-packages\gym\utils\passive_env_checker.py:219: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(
C:\Users\Mitko\.conda\envs\neat_test3\lib\site-packages\gym\utils\passive_env_checker.py:289: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


Population's average fitness: -10.19429 stdev: 6.57282
Best fitness: -3.70000 - size: (18, 300) - species 234 - id 40820
Population's average fitness: -10.19429 stdev: 6.57282
Best fitness: -3.70000 - size: (18, 300) - species 234 - id 40820
Average adjusted fitness: 0.601
Average adjusted fitness: 0.601
Mean genetic distance 3.185, standard deviation 0.344
Mean genetic distance 3.185, standard deviation 0.344
Population of 428 members in 49 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
   161   36    11     -4.0    0.621     5
   181   28     8     -4.0    0.344    14
   182   27    13     -4.0    0.728     5
   184   26     8     -4.0    0.615    11
   185   26    12     -3.7    0.704    13
   188   24    10     -4.0    0.600     9
   191   22     8     -4.0    0.706    12
   192   20    11     -3.7    0.638     6
   193   20     8     -4.0    0.429    14
   196   19     9     -4.0    0.485    11
   197   16    11     -4.0    0.641     4


 ****** Running generation 125 ****** 


 ****** Running generation 125 ****** 

Population's average fitness: -10.28304 stdev: 6.53457
Best fitness: -3.40000 - size: (19, 306) - species 203 - id 576
Population's average fitness: -10.28304 stdev: 6.53457
Best fitness: -3.40000 - size: (19, 306) - species 203 - id 576

Species 199 with 11 members is stagnated: removing it

Species 199 with 11 members is stagnated: removing it

Species 185 with 14 members is stagnated: removing it

Species 185 with 14 members is stagnated: removing it
Average adjusted fitness: 0.572
Average adjusted fitness: 0.572
Mean genetic distance 3.158, standard deviation 0.352
Mean genetic distance 3.158, standard deviation 0.352
Population of 489 members in 48 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
   161   38    15     -4.0    0.790     7
   182   29    12     -4.0    0.651     7
   184   28    11     -4.0    0.612    13
   188   26    14     -4.0    0.723  

...


 ****** Running generation 165 ****** 


 ****** Running generation 165 ****** 

Population's average fitness: -10.08760 stdev: 6.58333
Best fitness: -3.40000 - size: (27, 315) - species 283 - id 17264
Population's average fitness: -10.08760 stdev: 6.58333
Best fitness: -3.40000 - size: (27, 315) - species 283 - id 17264

Species 269 with 15 members is stagnated: removing it

Species 269 with 15 members is stagnated: removing it

Species 253 with 9 members is stagnated: removing it

Species 253 with 9 members is stagnated: removing it
Average adjusted fitness: 0.557
Average adjusted fitness: 0.557
Mean genetic distance 3.171, standard deviation 0.362
Mean genetic distance 3.171, standard deviation 0.362
Population of 491 members in 42 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
   251   30     9     -4.0    0.500     8
   252   30    24     -4.0    0.578    12
   254   29     2    -18.4    0.084     7
   256   29    14     -4.0    0.611


 ****** Running generation 167 ****** 


 ****** Running generation 167 ****** 

Population's average fitness: -10.68377 stdev: 6.74024
Best fitness: -3.40000 - size: (27, 304) - species 251 - id 18070
Population's average fitness: -10.68377 stdev: 6.74024
Best fitness: -3.40000 - size: (27, 304) - species 251 - id 18070

Species 276 with 15 members is stagnated: removing it

Species 276 with 15 members is stagnated: removing it

Species 265 with 16 members is stagnated: removing it

Species 265 with 16 members is stagnated: removing it

Species 256 with 16 members is stagnated: removing it

Species 256 with 16 members is stagnated: removing it
Average adjusted fitness: 0.518
Average adjusted fitness: 0.518
Mean genetic distance 3.127, standard deviation 0.377
Mean genetic distance 3.127, standard deviation 0.377
Population of 509 members in 36 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
   251   32    17     -3.4    0.563    10
   252 


 ****** Running generation 169 ****** 


 ****** Running generation 169 ****** 

Population's average fitness: -10.33561 stdev: 6.73919
Best fitness: -3.40000 - size: (18, 289) - species 281 - id 18897
Population's average fitness: -10.33561 stdev: 6.73919
Best fitness: -3.40000 - size: (18, 289) - species 281 - id 18897

Species 273 with 15 members is stagnated: removing it

Species 273 with 15 members is stagnated: removing it
Average adjusted fitness: 0.543
Average adjusted fitness: 0.543
Mean genetic distance 3.103, standard deviation 0.370
Mean genetic distance 3.103, standard deviation 0.370
Population of 492 members in 37 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
   251   34    15     -4.0    0.557    12
   252   34    14     -3.7    0.511     3
   254   33     2    -19.2    0.045    11
   258   32    13     -3.4    0.548    13
   262   31    12     -3.7    0.493    14
   263   29    11     -4.0    0.451    12
   270   22     9


 ****** Running generation 171 ****** 


 ****** Running generation 171 ****** 

Population's average fitness: -10.21748 stdev: 6.50342
Best fitness: -3.40000 - size: (25, 300) - species 304 - id 19691
Population's average fitness: -10.21748 stdev: 6.50342
Best fitness: -3.40000 - size: (25, 300) - species 304 - id 19691

Species 258 with 10 members is stagnated: removing it

Species 258 with 10 members is stagnated: removing it
Average adjusted fitness: 0.568
Average adjusted fitness: 0.568
Mean genetic distance 3.087, standard deviation 0.368
Mean genetic distance 3.087, standard deviation 0.368
Population of 488 members in 39 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
   251   36    12     -4.0    0.494    14
   252   36    13     -3.7    0.640     5
   254   35     2    -18.4    0.072    13
   263   31    13     -4.0    0.686    14
   270   24    13     -4.0    0.862     9
   275   22    17     -4.0    0.651     1
   277   21     9


Best genome:
Key: 20176
Fitness: -3.4000000000000017
Nodes:
	0 DefaultNodeGene(key=0, bias=0.335322634165237, response=1.048966419075242, activation=square, aggregation=sum)
	1 DefaultNodeGene(key=1, bias=0.9826676281209179, response=2.380258117396303, activation=relu, aggregation=sum)
	2 DefaultNodeGene(key=2, bias=-1.221599280032787, response=1.0331971512709701, activation=sigmoid, aggregation=sum)
	3 DefaultNodeGene(key=3, bias=-0.3746791592959453, response=2.567482598140332, activation=relu, aggregation=sum)
	4 DefaultNodeGene(key=4, bias=2.273041670206397, response=2.130243049815893, activation=hat, aggregation=sum)
	5 DefaultNodeGene(key=5, bias=0.5988101290236211, response=0.5972851911454515, activation=exp, aggregation=sum)
	6 DefaultNodeGene(key=6, bias=-0.6127584409797755, response=1.0, activation=inv, aggregation=sum)
	7 DefaultNodeGene(key=7, bias=3.666643372570265, response=0.9098639856035162, activation=relu, aggregation=sum)
	8 DefaultNodeGene(key=8, bias=0.981610084551

In [ ]:
visualize.plot_stats(stats, ylog=False, view=True)
visualize.plot_species(stats, view=True)

In [ ]:
visualize.draw_net(config=config, genome=winner)


In [ ]:
import neat

# Load configuration.
config_filename = 'config-feedforward_v01'
config = neat.Config(neat.DefaultGenome, 
                     neat.DefaultReproduction, 
                     neat.DefaultSpeciesSet, 
                     neat.DefaultStagnation, 
                     config_filename)

config.genome_config.add_node_mutation_prob = 0.03
config.genome_config.add_connection_mutation_prob = 0.05

# Print the parsed parameters to debug
print("Initial connection type:", config.genome_config.initial_connection)
print("Allowed connectivity options:", config.genome_config.allowed_connectivity)
print("Activation options:", config.genome_config.activation_options)


### ChatGPT

In [ ]:
import neat
import gym
import gym_sokoban
import pyglet
import numpy as np

import time
import logging
from neat.reporting import StdOutReporter

# Custom rendering setup if gym's rendering is not available
class Viewer:
    def __init__(self, width, height):
        self.window = pyglet.window.Window(width, height)
        self.image = None
        self.window.on_draw = self.on_draw

    def render(self, image):
        self.image = pyglet.image.ImageData(image.shape[1], image.shape[0], 'RGB', image.tobytes(), pitch=image.shape[1] * -3)
        self.window.dispatch_event('on_draw')

    def on_draw(self):
        if self.image:
            self.window.clear()
            self.image.blit(0, 0)

# Initialize logging
logging.basicConfig(filename='neat_log.txt', level=logging.INFO, format='%(message)s')

# Custom reporter class
class CustomReporter(StdOutReporter):
    def __init__(self, show_species_detail):
        super().__init__(show_species_detail)
        self.start_time = time.time()
    
    def end(self):
        runtime = time.time() - self.start_time
        logging.info(f'Total runtime: {runtime:.2f} seconds')

    def post_evaluate(self, config, population, species_set, best_genome):
        super().post_evaluate(config, population, species_set, best_genome)
        
        # Log population's average fitness
        total_fitness = sum(genome.fitness for genome in population.values())
        avg_fitness = total_fitness / len(population)
        logging.info(f'Population\'s average fitness: {avg_fitness}')
        
        # Log adjusted fitness score
        adjusted_fitness = []
        for species_id, species in species_set.species.items():
            for genome_id in species.members:
                genome = population[genome_id]
                adjusted_fitness.append(genome.fitness / len(species.members))
        avg_adjusted_fitness = sum(adjusted_fitness) / len(adjusted_fitness)
        logging.info(f'Population\'s average adjusted fitness: {avg_adjusted_fitness}')
        
        # Log best genome information
        logging.info(f'\nBest genome:\nKey: {best_genome.key}\nFitness: {best_genome.fitness}')
        logging.info(f'Nodes:')
        for node_key, node in best_genome.nodes.items():
            logging.info(f'\t{node_key} {node}')
        logging.info(f'Connections:')
        for conn_key, conn in best_genome.connections.items():
            logging.info(f'\t{conn_key} {conn}')
        logging.info(f'Timestamp: {time.strftime("%Y-%m-%d %H:%M:%S")}')


def process_observation(environment, obs):
    # Convert the observation to RGB frame or custom observation
    arr_walls, arr_goals, arr_boxes, arr_player = environment.render(mode='raw')

    # Initialize the combined array with walls (1s)
    combined = np.ones_like(arr_walls)
    
    # Set empty fields (0s)
    combined[arr_walls == 0] = 0
    
    # Set targets (3s)
    combined[arr_goals == 1] = 3
    
    # Set boxes (2s)
    combined[arr_boxes == 1] = 2
    
    # Set boxes on targets (4s)
    combined[(arr_boxes == 1) & (arr_goals == 1)] = 4
    
    # Set player position (5s)
    combined[arr_player == 1] = 5

    # Flatten the array
    flat_array = combined.flatten()
    
    return flat_array


def process_state(state):
    # Processes the initial state of env.reset()

    # Initialize the combined array with walls (0s)
    combined = np.ones_like(state[0])
    
    # Set empty fields (1s)
    combined[state[0] == 0] = 0

    # Set targets (3s)
    combined[state[1] == 1] = 3

    # Set boxes (2s)
    combined[state[2] == 1] = 2

    # Set boxes on targets (4s)
    combined[(state[2] == 1) & (state[1] == 1)] = 4

    # Set player position (5s)
    combined[state[3] == 1] = 5

    # Flatten the array
    flat_array = combined.flatten()
    
    return flat_array


# Start the game
env = gym.make('Sokoban-small-v1')
env.reset()

# Optional viewer setup
viewer = Viewer(160, 160)  # Adjust the size according to your environment
ACTION_LOOKUP = env.unwrapped.get_action_lookup()

# Define episode and timestep parameters
num_episodes = 1
timesteps_per_episode = 40

def map_action(action_prob):
    return np.argmax(action_prob)

def eval_genomes(genomes, config):
    global num_episodes, timesteps_per_episode

    # For each genome
    for genome_id, genome in genomes:
        # Generate the neural network based on the config provided
        net = neat.nn.FeedForwardNetwork.create(genome, config)
    
        # Define the initial fitness of the genome
        genome.fitness = 0.0
    
        # Define episodes rewards (list) idea is to keep the fitness scores of all episodes and take the max
        episodes_rewards = []
        
        # For each episode
        for episode in range(num_episodes):
            # Episode reward = 0  
            ep_reward = 0
            
            # Reset the game state to the initial phase
            initial_state = env.reset()

            # Map inputs suitable for the Neural Network (initial state of the game as flatten array, e.g. with length 49, created by the initial 7x7 grid)
            # Used as an input layer in the Neural Network
            initial_inputs = process_state(initial_state)
            
            game_state_after_step = initial_inputs
            
            # For each step
            for step in range(timesteps_per_episode):
                # Calculate probabilities for each output to be selected
                action_prob = net.activate(game_state_after_step)
                
                # Select the action, based on the output's probabilities of being selected
                action = map_action(action_prob)
                
                # Make the move in the game and output game state + info + reward
                observation, reward, done, info = env.step(action)
                
                game_state_after_step = process_observation(environment=env, obs=observation)

                # Image stuff
                image = env.render(mode='rgb_array')
                viewer.render(image)

                # Print info
                print(ACTION_LOOKUP[action], reward, done, info)

                # Populate the episode reward
                if not done:
                    ep_reward += reward
                else:
                    print(f"Game finished in {step+1} steps")
                    break
    
            # Add the accumulated rewards for this episode into a list (episodes_rewards)   
            episodes_rewards.append(ep_reward)

        # Choose the best performance on an episode for the genome
        genome.fitness = max(episodes_rewards)

        print("All episodes finished. Closing window.")
        viewer.window.close()  # Close the Pyglet window explicitly

# Load configuration.
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     'config-feedforward')

# Create the population, which is the top-level object for a NEAT run.
p = neat.Population(config)

# Add a stdout reporter to show progress in the terminal.
p.add_reporter(neat.StdOutReporter(True))
p.add_reporter(CustomReporter(True))
p.add_reporter(neat.StatisticsReporter())

# Run until a solution is found. The number indicates the max number of generations to be produced
winner = p.run(eval_genomes, 5)

# Display the winning genome.
print('\nBest genome:\n{!s}'.format(winner))

# Show output of the most fit genome against training data.
print('\nOutput:')
winner_net = neat.nn.FeedForwardNetwork.create(winner, config)

# Start the Pyglet event loop to keep the window open
pyglet.app.run()


### BACKUP

In [ ]:
# Start the game

env = gym.make('Sokoban-small-v1')
# generate the level in the initial stage (env.reset) 
env.reset()


# # OPTIONAL

# viewer = Viewer(160, 160)  # Adjust the size according to your environment

# ACTION_LOOKUP = env.unwrapped.get_action_lookup()

# Define episode and timestep parameters
num_episodes = 1
timesteps_per_episode = 40

current_episode = 0
current_timestep = 0

min_reward = -10


def eval_genomes(genomes, config)
# FOR EACH GENOME
    for genome_id, genome in genomes:
        
        env = gym.make()
    # net = neat.nn.FeedForwardNetwork.create(genome, config)
    # DEF INITIAL GENOME FITNESS = 0
    
    
    # EPISODES REWARDS = [] IDEA IS TO KEEP THE FITNESS SCORES OF ALL EPISODES AND THEN TAKE THE MAX
    
    # FOR EACH EPISODE
        # Episode reward = 0  
        # env.reset()
            
        # FOR EACH STEP
            # ACTION - GENERATED BY THE GENOME
            # RANDOM ACTION
            action = env.action_space.sample()
            
            # MAKE THE MOVE IN THE GAME
            # OUTPUT GAME STATE AFTER THE STEP WITH INFO + REWARD            
            observation, reward, done, info = env.step(action)
            
            
            # IMAGE STUFF
            image = env.render(mode='rgb_array')
            viewer.render(image)
            
            # PRINT INFO
            print(ACTION_LOOKUP[action], reward, done, info)

            # POPULATE THE Episode reward +=
            # if not done:
                # reward += MIN REWARD
                # current_timestep += 1

            # if done:
                # current_timestep = 0
                # current_episode += 1

                # OR BREAK

        # EPISODES REWARDS APPEND episode reward   
        
    
    # GENOME.FITNESS = max(EPISODE REWARDS)        
                

        

# # Load configuration.
# config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
#                      neat.DefaultSpeciesSet, neat.DefaultStagnation,
#                      'config-feedforward')

# # Create the population, which is the top-level object for a NEAT run.
# p = neat.Population(config)

# # Add a stdout reporter to show progress in the terminal.
# p.add_reporter(neat.StdOutReporter(False))

# # Run until a solution is found.
# winner = p.run(eval_genomes, 5)

# # Display the winning genome.
# print('\nBest genome:\n{!s}'.format(winner))

# # Show output of the most fit genome against training data.
# print('\nOutput:')
# winner_net = neat.nn.FeedForwardNetwork.create(winner, config)    

### Game logic

In [ ]:
import gym
import gym_sokoban
import pyglet
from pyglet import clock
import numpy as np

## Custom rendering setup if gym's rendering is not available
class Viewer:
    def __init__(self, width, height):
        self.window = pyglet.window.Window(width, height)
        self.image = None
        self.window.on_draw = self.on_draw

    def render(self, image):
        self.image = pyglet.image.ImageData(image.shape[1], image.shape[0], 'RGB', image.tobytes(), pitch=image.shape[1] * -3)
        self.window.dispatch_event('on_draw')

    def on_draw(self):
        if self.image:
            self.window.clear()
            self.image.blit(0, 0)

env = gym.make('Sokoban-small-v1')
# generate the level in the initial stage (env.reset) 
env.reset()

print("Room Fixed")
print(env.room_fixed)
print(type(env.room_fixed))
print(env.room_fixed.shape)
print()
print(env.room_state)
print()
print(env.box_mapping)
print()


viewer = Viewer(160, 160)  # Adjust the size according to your environment

ACTION_LOOKUP = env.unwrapped.get_action_lookup()

# Define episode and timestep parameters
num_episodes = 2
timesteps_per_episode = 100

current_episode = 0
current_timestep = 0

def update_environment(dt):
    global current_episode, current_timestep, num_episodes, timesteps_per_episode

    if current_episode < num_episodes:
        if current_timestep < timesteps_per_episode:
            # RANDOM ACTION
            action = env.action_space.sample()
            observation, reward, done, info = env.step(action)
            
            
            
            image = env.render(mode='rgb_array')
            viewer.render(image)

            print(ACTION_LOOKUP[action], reward, done, info)

            if done:
                print(f"Episode finished after {current_timestep + 1} timesteps")
                current_timestep = 0
                current_episode += 1
                env.reset()
            else:
                current_timestep += 1
        else:
            current_episode += 1
            current_timestep = 0
            env.reset()
    else:
        print("All episodes finished. Closing window.")
        viewer.window.close()  # Close the Pyglet window explicitly

# Increase the frequency to match rendering needs (e.g., 60Hz)
clock.schedule_interval(update_environment, 1/60.0)

pyglet.app.run()
